In [1]:
import mfem.ser as mfem
import numpy as np
from bfp import *
import math
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from glvis import glvis
import os
import h5py

In [2]:
def create_2D_mesh(nx, ny, x_start, x_end, y_start, y_end):
    """Creates a 2D mesh with the specified intervals and coordinate ranges.

    Args:
        nx (int): Number of intervals in the x-direction.
        ny (int): Number of intervals in the y-direction.
        x_start (float): Starting x-coordinate.
        x_end (float): Ending x-coordinate.
        y_start (float): Starting y-coordinate.
        y_end (float): Ending y-coordinate.

    Returns:
        mesh: The updated mesh with vertex coordinates set accordingly.
    """

    x_coords = np.linspace(x_start, x_end, nx + 1)
    y_coords = np.linspace(y_start, y_end, ny + 1)
    
    mesh = mfem.Mesh(nx, ny, "QUADRILATERAL", True, 0.0, 0.0)
    verts = mesh.GetVertexArray()

    k = 0
    for j in range(ny + 1):
        for i in range(nx + 1):
            verts[k][0] = x_coords[i]
            verts[k][1] = y_coords[j]
            k += 1

    target_dir = os.path.join(os.getcwd(), 'mesh', 'usr')
    file_name = f'{nx}x{ny}_2D.mesh'
    file_path = os.path.join(target_dir, file_name)
    
    if not os.path.exists(target_dir):
        os.makedirs(target_dir)
        print(f"Directory '{target_dir}' was created.")

    if not os.path.exists(file_path):
        mesh.Print(file_path)
        print(f"File '{file_path}' was successfully created.")
    else:
        print(f"File '{file_path}' already exists.")

    return mesh

def read_data(nE):
    """Read cross-section and stopping power data from an HDF5 file.

    Args:
        nE (int): Number of energy groups.

    Returns:
        tuple: A tuple containing:
            - E_arr (numpy.ndarray): Energy array.
            - E_grid_arr (numpy.ndarray): Energy grid array.
            - xs_t_arr (numpy.ndarray): Total cross-section array.
            - xs_s_arr (numpy.ndarray): Scattering cross-section array.
            - S_arr (numpy.ndarray): Stopping power array.
    """

    data_file = "data.h5"

    with h5py.File(data_file, "r") as f:
        E_arr = f[f"E_{nE}"][:]
        E_grid_arr = f[f"E_grid_{nE}"][:]
        xs_t_arr = f[f"xs_t_{nE}"][:]
        xs_s_arr = f[f"xs_s_{nE}"][:]
        S_arr    = f[f"S_{nE}"][:]

    return E_arr, E_grid_arr, xs_t_arr, xs_s_arr, S_arr

def gauss_legendre_dirs(N_dir):
    """Compute Gauss-Legendre quadrature points and weights for discrete ordinates.

    This function computes the Gauss-Legendre quadrature points (mu_i) and weights (w_i)
    for the specified number of discrete ordinates using NumPy's polynomial module.

    Args:
        N_dir (int): The number of discrete ordinates.

    Returns:
        tuple: A tuple containing two lists:
            - mu (list of float): The Gauss-Legendre quadrature points.
            - w (list of float): The corresponding Gauss-Legendre quadrature weights.
    """

    mu, w = np.polynomial.legendre.leggauss(N_dir)
    mu_tr = 0.5 * (mu + 1)
    w_tr = 0.5 * w
    return mu_tr, w_tr


class EnergyDependentCoefficient(mfem.PyCoefficient):
    def __init__(self, data, E_start=None, E_end=None):
        super(EnergyDependentCoefficient, self).__init__()
        if isinstance(data, (int, float)):
            self.constant = True
            self.constant_value = float(data)
        else:
            self.constant = False
            self.data = data
            self.E_start = E_start
            self.E_end = E_end
    def EvalValue(self,ip):
        if self.constant:
            return self.constant_value
        y = ip[1] 
        E = self.E_start + y * (self.E_end - self.E_start)
        n_groups = len(self.data)
        group = min(n_groups - 1, int((E - self.E_start) / (self.E_end - self.E_start) * n_groups))
        return float(self.data[group])
        
class InflowCoefficient(mfem.PyCoefficient):
    def __init__(self, in_flux):
        super(InflowCoefficient, self).__init__()
        self.in_flux = in_flux

    def EvalValue(self, ip):
            return self.in_flux
    

class VelocityCoefficient(mfem.VectorPyCoefficientBase):
    def __init__(self, mu, S_coeff):
        mfem.VectorPyCoefficientBase.__init__(self, 2, 0)
        self.mu = mu
        self.S_coeff = S_coeff
    def _EvalPy(self, V, ip):
        V[0] = self.mu
        V[1] = self.S_coeff.EvalValue(ip)
        return V



In [3]:
# Set parameters
nx = 20
nE = 50
x_start = 0.0
x_end = 0.3
E_start = 1
E_end = 0.01
N_ang = 8
order = 1
inflow = 100

# Define Mesh
mesh = mfem.Mesh("fixed.mesh", 1, 1)
dim = mesh.Dimension()
# Define fec and fes
# btype = 0-GaussLegendre, 1 - GaussLobatto
fec = mfem.DG_FECollection(order, dim, btype=1)
fes = mfem.FiniteElementSpace(mesh, fec)
print("Number of DG DOFs:", fes.GetTrueVSize())

# Define Coefficients
E_arr, E_grid_arr, xs_t_arr, xs_s_arr, S_arr = read_data(nE)
xs_t_arr /= 100
dS_dE_arr = compute_S_derivative(E_arr, S_arr)
S_coeff = EnergyDependentCoefficient(S_arr, E_start, E_end)
inf_coeff = InflowCoefficient(inflow)
xs_t_coeff = EnergyDependentCoefficient(xs_t_arr, E_start, E_end)
xs_s_coeff = EnergyDependentCoefficient(xs_s_arr, E_start, E_end)
dS_dE_coeff = EnergyDependentCoefficient(dS_dE_arr, E_start, E_end)

mu_vals, w_vals = gauss_legendre_dirs(N_ang)
inflow_bdr_attr = [8]

Number of DG DOFs: 4000


In [4]:
psi_mu_list = []

for i in range (N_ang):
    mu = mu_vals[i]
    w = w_vals[i]
    print("  Solving for mu =", mu)
    v_coeff = VelocityCoefficient(mu, S_coeff)

    # Assemble Bilinear and Linear Forms
    a = mfem.BilinearForm(fes)
    a.AddDomainIntegrator(mfem.ConvectionIntegrator(v_coeff, 1.0))
    a.AddInteriorFaceIntegrator(mfem.DGTraceIntegrator(v_coeff, 1.0, -1.0))
    a.AddDomainIntegrator(mfem.MassIntegrator(xs_t_coeff))
    a.AddDomainIntegrator(mfem.MassIntegrator(dS_dE_coeff))
    #a.AddInteriorFaceIntegrator(mfem.TransposeIntegrator(mfem.DGTraceIntegrator(v_coeff, -1.0, 0.5)))
    a.AddBdrFaceIntegrator(mfem.TransposeIntegrator(mfem.DGTraceIntegrator(v_coeff, -1.0, 0.5)))
    a.Assemble()
    a.Finalize()
    A = a.SpMat()

    # Set up the linear form b for the RHS
    b = mfem.LinearForm(fes)
    #b.AddBdrFaceIntegrator(mfem.DGDirichletLFIntegrator(inf_coeff, v_coeff, 1.0, -0.5))
    b.Assemble()

    psi = mfem.GridFunction(fes)
    psi.Assign(1.0)
    prec = mfem.GSSmoother(A)
    solver = mfem.GMRESSolver()

    solver.SetPreconditioner(prec)
    solver.SetRelTol(1e-12)
    solver.SetAbsTol(1e-12)
    solver.SetMaxIter(500)
    solver.SetKDim(30)
    solver.SetPrintLevel(0)
    solver.SetOperator(A)
    solver.Mult(b, psi)
    
    print("    GMRES (mu =", mu, "): iterations =", solver.GetNumIterations(),
          "final norm =", solver.GetFinalNorm())
    
    # Append the angular solution for mu
    psi_mu_list.append((mu, w, psi))
    
    # Save each angular solution and corresponding mesh.
    psi.Save("psi_mu_list{:.3f}.gf".format(mu))
phi_new = mfem.GridFunction(fes)
phi_new.Assign(0.0)

for mu, w, psi in psi_mu_list:
    phi_new.Add(w, psi)

phi_new.Save("phi_newww.gf")


  Solving for mu = 0.019855071751231912
    GMRES (mu = 0.019855071751231912 ): iterations = 10 final norm = 1.385646187393429e-11
  Solving for mu = 0.10166676129318664
    GMRES (mu = 0.10166676129318664 ): iterations = 10 final norm = 1.385646187393429e-11
  Solving for mu = 0.2372337950418355
    GMRES (mu = 0.2372337950418355 ): iterations = 10 final norm = 1.385646187393429e-11
  Solving for mu = 0.4082826787521751
    GMRES (mu = 0.4082826787521751 ): iterations = 10 final norm = 1.385646187393429e-11
  Solving for mu = 0.591717321247825
    GMRES (mu = 0.591717321247825 ): iterations = 10 final norm = 1.385646187393429e-11
  Solving for mu = 0.7627662049581645
    GMRES (mu = 0.7627662049581645 ): iterations = 10 final norm = 1.385646187393429e-11
  Solving for mu = 0.8983332387068134
    GMRES (mu = 0.8983332387068134 ): iterations = 10 final norm = 1.385646187393429e-11
  Solving for mu = 0.9801449282487681
    GMRES (mu = 0.9801449282487681 ): iterations = 10 final norm = 1.

In [5]:
glvis((mesh, phi_new), 1200, 1200)

_GlvisWidgetCore(data_str='solution\nMFEM mesh v1.0\n\n#\n# MFEM Geometry Types (see fem/geom.hpp):\n#\n# POIN…

In [6]:
mesh1 = mfem.

SyntaxError: invalid syntax (2170138426.py, line 1)